# DOME Registry Pre-Repair Analysis

This notebook is designed to analyze the DOME registry data, search for specific papers, and cross-reference user metadata before any DSW (Daily Study Workflow) repair operations are performed. It provides tools to verify the state of annotations and curator mappings.


In [3]:
import requests
import time
import pandas as pd

def fetch_epmc_metadata(pmid):
    """
    Fetches Title, Authors, PubYear, and DOI from Europe PMC API for a given PMID.
    """
    if not pmid or str(pmid) == 'nan':
        return None, None, None, None
        
    url = "https://www.ebi.ac.uk/europepmc/webservices/rest/search"
    params = {
        'query': f'EXT_ID:{pmid} SRC:MED',
        'format': 'json',
        'resultType': 'core'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        result_list = data.get('resultList', {}).get('result', [])
        
        if result_list:
            top_result = result_list[0]
            title = top_result.get('title', '')
            authors = top_result.get('authorString', '')
            pub_year = top_result.get('pubYear', '')
            doi = top_result.get('doi', '')
            return title, authors, pub_year, doi
            
    except Exception as e:
        print(f"Error fetching PMID {pmid}: {e}")
        
    return None, None, None, None

# Load the TSV
tsv_file = "Dome-Recommendations-Annotated-Articles_20250202.tsv"
print(f"Loading {tsv_file}...")
df_recs = pd.read_csv(tsv_file, sep='\t')

# Ensure columns exist
new_cols = ['EPMC_title', 'EPMC_authors', 'EPMC_pub_year', 'EPMC_doi']
for col in new_cols:
    if col not in df_recs.columns:
        df_recs[col] = None

# Iterate and Enrich
print("Enriching data with Europe PMC API...")
total = len(df_recs)

for index, row in df_recs.iterrows():
    pmid = row.get('PMID')
    
    # Skip if already filled (optional, but good for retries)
    # or if PMID is missing
    if pd.isna(pmid):
        continue
        
    print(f"Processing {index + 1}/{total}: PMID {pmid}", end='\r')
    
    title, authors, year, doi = fetch_epmc_metadata(pmid)
    
    df_recs.at[index, 'EPMC_title'] = title
    df_recs.at[index, 'EPMC_authors'] = authors
    df_recs.at[index, 'EPMC_pub_year'] = year
    df_recs.at[index, 'EPMC_doi'] = doi
    
    # Be nice to the API
    time.sleep(0.1)

print("\nEnrichment complete.")
df_recs.head()

Loading Dome-Recommendations-Annotated-Articles_20250202.tsv...
Enriching data with Europe PMC API...
Processing 188/188: PMID 24977146
Enrichment complete.


,Informazioni cronologiche,PMID,Journal name,Publication year,DOME version,Provenance,Dataset splits,Redundancy between data splits,Availability of data,Algorithm,...,Evaluation method,Performance measures,Comparison,Confidence,Availability of evaluation,Indirizzo email,EPMC_title,EPMC_authors,EPMC_pub_year,EPMC_doi
0,3/23/2022 16:36:18,33465072,PLoS Comput Biol.,2021,1.0,"yes, N_pos: 13128 N_neg: 32766",5-fold cross-validation,Correlation Feature Selection is used.,"yes, described at S1 table: https://deposition...","Naive Bayes, SVM, Random Forests",...,5-fold cross validation,ROC curves and AUC values,NaN,NaN,NaN,andrewhatos@gmail.com,Genome-wide prediction of topoisomerase IIβ bi...,"Martínez-García PM, García-Torres M, Divina F,...",2021,10.1371/journal.pcbi.1007814
1,3/28/2022 0:44:11,33679869,Front Genet.,2021,1.0,public databases: 901 samples from The Cancer ...,10-fold cross validation,NaN,"no, they claim: ""Publicly available datasets w...","Spathial, Random forest, LASSO",...,10-fold cross validation,"AUCs for 2-, 3-, and 5-year OS were 0.527, 0.5...",no,no,no,andrewhatos@gmail.com,Construction and Comprehensive Analyses of a M...,"Sun S, Fei K, Zhang G, Wang J, Yang Y, Guo W, ...",2020,10.3389/fgene.2020.617174
2,3/28/2022 16:40:10,34419924,EBioMedicine.,2021,1.0,samples from Stanford Health Care and Stanford...,The final analysis included for discovery coho...,NaN,Conatact (hoganca@stanford.edu) is provided ri...,gradient boosted decision trees and random for...,...,novel experiments.,"AUC, sensitivity, specificity",costeffective comare to PCR tests and could be...,NaN,"yes, https://github.com/stanfordmlgroup/influe...",andrewhatos@gmail.com,Nasopharyngeal metabolomics and machine learni...,"Hogan CA, Rajpurkar P, Sowrirajan H, Phillips ...",2021,10.1016/j.ebiom.2021.103546
3,3/29/2022 23:01:05,34112769,Nat Commun .,2021,1.0,Clinical data collected by the authors N_neg 3...,N_pos: Discovery 227 and Validation 77,NaN,All data included in this study is available u...,composite model with several stepp with diffe...,...,independent dataset,"Accuracy (94.81% for the C4 prediction model, ...",no,no,no,andrewhatos@gmail.com,A new molecular classification to drive precis...,"Soret P, Le Dantec C, Desvaux E, Foulquier N, ...",2021,10.1038/s41467-021-23472-7
4,3/28/2022 12:23:43,32915751,IEEE J Biomed Health Inform.,2020,1.0,two public COVID-19 CT datasets: - https:/...,no,no,preprocessed data: https://drive.google.com/fi...,"deep learning, novel approach",...,four-fold cross-validation,"Accuracy, F1 score, Sensitivity, Precision, AUC",firstly comparision to COVID-Net method https:...,outperforming the original COVID-Net trained o...,no,andrewhatos@gmail.com,Contrastive Cross-Site Learning With Redesigne...,"Wang Z, Liu Q, Dou Q.",2020,10.1109/jbhi.2020.3023246


In [ ]:
import json

# 1. Integrate User OIDs
users_file = "dome_users_20260130.json"
try:
    print(f"Loading users from {users_file}...")
    with open(users_file, 'r', encoding='utf-8') as f:
        users_data = json.load(f)
        
    # Create Email -> OID mapping
    email_to_oid = {}
    for u in users_data:
        email = u.get('email')
        oid = u.get('_id', {}).get('$oid')
        if email and oid:
            email_to_oid[email.strip()] = oid
            
    # Apply to DataFrame
    # Target column is 'Indirizzo email' based on file inspection
    if 'Indirizzo email' in df_recs.columns:
        df_recs['User_OID'] = df_recs['Indirizzo email'].apply(lambda x: email_to_oid.get(str(x).strip(), 'Unknown'))
        print("User OIDs mapped successfully.")
    else:
        print("Warning: 'Indirizzo email' column not found in TSV.")

except Exception as e:
    print(f"Error mapping users: {e}")

# 2. Reorder columns (EPMC info after PMID)
cols = list(df_recs.columns)
if 'PMID' in cols:
    pmid_idx = cols.index('PMID')
    
    # Remove EPMC cols if they are in the list currently
    active_new_cols = [c for c in new_cols if c in cols]
    for col in active_new_cols:
        cols.remove(col)
        
    # Insert them back after PMID
    for i, col in enumerate(active_new_cols):
        cols.insert(pmid_idx + 1 + i, col)
        
    df_recs = df_recs[cols]

# 3. Save (Overwriting/Creating the single enriched file)
output_file = "Dome-Recommendations-Annotated-Articles_20250202_Enriched.tsv"
df_recs.to_csv(output_file, sep='\t', index=False)
print(f"Saved enriched data with User OIDs to {output_file}")

Saved enriched data to Dome-Recommendations-Annotated-Articles_20250202_Enriched.tsv
